In [239]:
import jax.numpy as np
from jax import grad, jacfwd, jacrev
import pandas as pd
import collections
from sklearn.preprocessing import PolynomialFeatures
from scipy.optimize import root
import matplotlib.pyplot as plt
import itertools
import warnings

jax jacfwd seems to give same answers as grad_2d_polynomial only different shape. Check input 3x2 matrix into grad_2d_polynomial and into gradfwd. 0 containing rows in gradfwd might be dimensions not adding up

In [24]:
model_coef = pd.read_csv('../hydropt/data/PACE_polynom_05.csv', index_col = 0)
model_coef = model_coef.values
model_powers = PolynomialFeatures(degree=5).fit([[1,1]]).powers_

In [231]:
def grad_2d_polynomial(x):
    '''
    gradient of 2 variable polynomial

    x: points at wich to evaluate the derivative; a nx2 array were n is number of wavebands
    '''
    c = model_coef
    p = model_powers
    # check if dimensions match of polynomial and x
    if c.shape[0] is not x.shape[0]:
        warnings.warn('matrix dimensions of x and c do not match!')
    # get derivative terms of polynomial features
    d_x1 = lambda x, p: p[0]*x[0]**(float(p[0]-1))*x[1]**p[1]
    d_x2 = lambda x, p: p[1]*x[1]**(float(p[1]-1))*x[0]**p[0]
    # evaluate terms at [x]
    ft = np.array([[d_x1(x,p), d_x2(x,p)] for (x,p) in zip(itertools.cycle([x.T]), p)])
    # dot derivative matrix with polynomial coefficients and get diagonal
    dx = np.array([np.dot(c,ft[:,0,:]).diagonal(),
                   np.dot(c,ft[:,1,:]).diagonal()])

    return dx

In [11]:
def _polynomial_features(x):
    x = x.T
    f = np.array([(x[0]**i)*(x[1]**j) for (i,j) in model_powers]).T

    return f

In [227]:
#ft_in = _polynomial_features(np.ones([2,2]))
def hydrolight_polynom(x): #, degree=5):
    '''
    Forward model using polynomial fit to Hydrolight simulations

    x[0]: total absorption at wavelength i
    x[1]: total backscatter at wavelength i

    returns Rrs
    '''
    # log absorption, backscatter
    x_log = np.log(x)
    x_log = x
    # get polynomial features
    #ft = PolynomialFeatures(degree=degree).fit_transform(x_log.T)
    ft = _polynomial_features(x_log.T)
    # get polynomial coefficients
    c = model_coef[5:7]
    # calculate log(Rrs)
    # SOMETHING GOES WRONG HERE
    log_rrs = np.dot(c, ft.T).diagonal()
    # calculate Rrs
    rrs = np.exp(log_rrs)

    return log_rrs

In [228]:
def polynom(x):
    c = model_coef[5:7]
    f = np.array([(x[0]**i)*(x[1]**j) for (i,j) in model_powers])
    out = np.dot(c, f).diagonal()
    
    return out 

In [229]:
jacfwd(hydrolight_polynom)(2*np.ones([2,2]))

DeviceArray([[[-1.0972822,  0.       ],
              [ 1.9201182,  0.       ]],

             [[ 0.       , -1.0633963],
              [ 0.       ,  1.8990856]]], dtype=float32)

In [230]:
grad_2d_polynomial(2*np.ones([2,2]))

DeviceArray([[-1.0972822, -1.0633963],
             [ 1.9201182,  1.8990856]], dtype=float32)

In [265]:
def fun(x):
    return np.array([x[0]  + 0.5 * (x[0] - x[1])**3 - 1.0,
            0.5 * (x[1] - x[0])**3 + x[1]])

In [289]:
def fun_2(x):
    return np.array([(x[0]  + 0.5 * (x[0] - x[1])**3 - 1.0)*2,
            1 * ((x[1] - x[0])**3 + x[1])])

In [307]:
def fun_3(x):
    return np.array([x[0]**2-x[1],
            x[0]*x[1]-3])

In [314]:
def fun_3_2(x):
    return np.array([(x[0]**2-x[1])/2,
            (x[0]*x[1]-3)/2])